In [1]:
import numpy as np

import tensorflow as tf

import pylab
%matplotlib inline

In [2]:
X_data = np.loadtxt('./coordinates1.csv', delimiter='$').astype('float64')
Y_data = np.loadtxt('./data_genotypes1.csv', delimiter='$').astype('float64')

In [5]:
assert(len(X_data) == len(Y_data))

N = X_data.shape[0]
k = X_data.shape[1]
M = Y_data.shape[1]
Y_data[Y_data < 0.5] = -1.0

In [6]:
with tf.device('/cpu:0'):
    X = tf.Variable(dtype=tf.float64, initial_value=X_data, trainable=False)
    Y = tf.Variable(dtype=tf.float64, initial_value=Y_data, trainable=False)
    
    F = tf.Variable(dtype=tf.float64, initial_value=np.random.normal(0.0, 0.1, (N, M)).astype('float64'),
                    trainable=True)

    l = tf.placeholder(shape=[], dtype=tf.float64)
    a = tf.placeholder(shape=[], dtype=tf.float64)

    K = a ** 2 * tf.exp(-tf.reduce_sum(((X[:, None] - X[None, :]) ** 2) / (2 * l ** 2), reduction_indices=[2])) + \
        0.00001 * tf.eye(N, dtype=tf.float64)

    p = tf.nn.sigmoid(F)
    Kinv_F = tf.matrix_solve(K, F)
    
    data_fit = -tf.reduce_sum(tf.log(1 + tf.exp(-Y * F)), reduction_indices=[0])
    prior_fit = -0.5 * tf.reduce_sum(F * Kinv_F, reduction_indices=[0])
    
    logL = tf.reduce_sum(data_fit + prior_fit)
    
    g =  (0.5 * (Y + 1) - p) - Kinv_F
    
    W = -p * (1 - p)
    lhs = -K[:, :, None] * W[None, :, :] - tf.eye(N, dtype=tf.float64)[:, :, None]
    rhs = tf.matmul(K, g)
    
    update = tf.matrix_solve(tf.transpose(lhs, [2, 0, 1]), tf.transpose(rhs, [1, 0])[:, :, None])[:, :, 0]
    update = tf.transpose(update)
    
    opt_op = F.assign(F - update)

In [7]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.01

In [12]:
with tf.Session(config=config) as sess:
    
    sess.run(tf.initialize_all_variables())
    
    for i in range(10):
        r = sess.run([opt_op, update, logL], {a: 1.0, l:10.0})
        print(r[2])
        
    r = F.eval()

-78274.7599115
-3385774.50737
-144891.666885
-inf
-inf
-inf
-inf


KeyboardInterrupt: 

In [9]:
r

array([[-0.39725825,  0.39605903, -0.39790925, ..., -0.39693404,
         0.39759144,  0.39621324],
       [ 0.39816345,  0.39780454, -0.39672318, ...,  0.39620898,
         0.39787283,  0.39727394],
       [-0.39826259, -0.39731617, -0.39678287, ..., -0.39590112,
         0.39626742,  0.39624388],
       ..., 
       [-0.39733744, -0.39914995,  0.39774144, ...,  0.39694304,
         0.39647871,  0.39698082],
       [-0.39724663, -0.39578777,  0.39656476, ..., -0.39741928,
        -0.39670453, -0.39726919],
       [-0.39679124, -0.39720114,  0.39785054, ...,  0.39794193,
        -0.39552965, -0.39744453]])

In [ ]:
W

In [ ]:
data_fit

In [ ]:
prior_fit

In [ ]:
tf.hessians

In [ ]:
update

In [ ]:
np.max(X_data)

In [ ]:
np.min(X_data)

In [ ]:
import time

In [ ]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.10

In [ ]:
x = tf.placeholder(shape=[20, 32, 32, 160], dtype=tf.float32)

W = tf.Variable(dtype=tf.float32, initial_value=np.random.random((1, 1, 160, 160)).astype('float32'))

op1 = tf.nn.conv2d(x, W, [1, 1, 1, 1], padding='SAME')
op2 = tf.reshape(tf.matmul(tf.reshape(x, [-1, 160]), W[0, 0]), [20, 32, 32, 160])

In [ ]:
times = []
with tf.Session(config=config) as sess:
    sess.run(tf.initialize_all_variables())
    for i in range(100):
        tic = time.time()
        sess.run(op1, feed_dict={x: np.random.random((20, 32, 32, 160))})
        times.append(time.time() - tic)
print(np.mean(times[10:]))

In [ ]:
times = []
with tf.Session(config=config) as sess:
    sess.run(tf.initialize_all_variables())
    for i in range(100):
        tic = time.time()
        sess.run(op2, feed_dict={x: np.random.random((20, 32, 32, 160))})
        times.append(time.time() - tic)
print(np.mean(times[10:]))